- добавить комментарии к ячейкам
- оформить все

Задание из видео Карпова [Разбор тестового задания на позицию junior аналитика данных](https://www.youtube.com/watch?v=3VWDh-gdiYg)

Описание данных:  
orders - таблица с информацией о заказах;  
order_id - уникальный идентификатор заказа;  
customer_id - уникальный идентификатор клиента;  
order_date - дата, когда заказ был создан;  
order_amount - cумма заказа;  
platform - платформа, через которую был сделан заказ (например, 'mobile', 'desktop');  
category - категория товара, который был заказан (например, 'smartphones', 'laptops').

1. Построй когортый анализ: выдели клиентов, совершивших первую покупку в течение определенного месяца (когортную дату) и вычисли размер каждой когорты.  
2. Рассчитай средний доход с клиента для каждой когорты за 10 дней с момента их первой покупки.  
3. Отобрази результаты по месяцам первых покупок, категориям товаров и платформам.  

Вывод должен содержать следующие поля:
- дата когорты;
- платформа;
- категория товара;
- размер когорты;
- средний доход с клиента.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [208]:
customer_ids = np.array([])
ids = []
for i in range(1, 1001):
    ids.append(i)
    customer_ids = np.append(customer_ids, np.random.choice(ids, (1, i // 50 + 1)))

In [209]:
customer_ids = customer_ids.astype(int)

In [210]:
# число заказов
N = customer_ids.size

In [211]:
order_dates = pd.date_range('2023-01-01', '2024-02-01', periods=N)

In [212]:
order_amounts = 111 + np.random.exponential(1, N) * 10000

In [213]:
platforms = np.random.choice(['mobile', 'desktop', 'shop'], N, p=[0.5, 0.3, 0.2])

In [214]:
categories = np.random.choice(['smartphones', 'laptops', 'watches'], N, p=[0.5, 0.3, 0.2])

In [215]:
df = pd.DataFrame({
    'order_id': range(1, N + 1), 
    'customer_id': customer_ids, 
    'order_date': order_dates, 
    'order_amount': order_amounts, 
    'platform': platforms, 
    'category': categories
})

In [216]:
df.dtypes

order_id                 int64
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform                object
category                object
dtype: object

In [217]:
df.order_id = df.order_id.astype('int32')

In [218]:
df.platform = df.platform.astype('category')

In [219]:
df.category = df.category.astype('category')

In [220]:
df.dtypes

order_id                 int32
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform              category
category              category
dtype: object

In [221]:
# df.to_pickle('data.pkl')

In [222]:
# времядату превращаем в дату
df['order_day'] = df.order_date.dt.to_period('D')

In [223]:
# месяц, в котором совершена первая покупка
df['first_order_month'] = (
    df
    .groupby('customer_id')['order_date']
    .transform('min')
    .dt.to_period('M')
)

In [224]:
# сколько дней прошло с первой покупки
df['day_from_start'] = (
    df['order_day'].astype(int) 
    - df.groupby('customer_id')['order_day'].transform('min').astype(int)
)
df['day_from_start'] = df['day_from_start'].astype('int32')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: casting period[D] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: casting period[D] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  after removing the cwd from sys.path.


In [225]:
# размер когорты каждого месяца
df.groupby('first_order_month').customer_id.nunique()

first_order_month
2023-01    212
2023-02    112
2023-03     91
2023-04     83
2023-05     61
2023-06     60
2023-07     49
2023-08     63
2023-09     46
2023-10     51
2023-11     44
2023-12     43
2024-01     38
Freq: M, Name: customer_id, dtype: int64

In [226]:
(
    df[df.day_from_start < 10]
    .pivot_table('order_amount', 'first_order_month', 'platform')[:12]
    .round()
    .astype(int)
    .style.background_gradient()
)

platform,desktop,mobile,shop
first_order_month,,,
2023-01,10902.000000,9264.000000,9859.000000
2023-02,9976.000000,8934.000000,8927.000000
2023-03,9021.000000,8868.000000,9927.000000
2023-04,11677.000000,9648.000000,7954.000000
2023-05,10993.000000,9454.000000,8875.000000
2023-06,8868.000000,8096.000000,11922.000000
2023-07,9913.000000,8168.000000,8106.000000
2023-08,12081.000000,10664.000000,11308.000000
2023-09,11688.000000,13784.000000,4942.000000


In [227]:
(
    df[df.day_from_start < 10]
    .pivot_table('order_amount', 'first_order_month', 'category')[:12]
    .round()
    .astype(int)
    .style.background_gradient()
)

category,laptops,smartphones,watches
first_order_month,,,
2023-01,8807.000000,10669.000000,9511.000000
2023-02,11693.000000,8146.000000,8984.000000
2023-03,8996.000000,9488.000000,8336.000000
2023-04,11480.000000,9176.000000,9551.000000
2023-05,8769.000000,9497.000000,11814.000000
2023-06,9250.000000,9512.000000,8726.000000
2023-07,6749.000000,7870.000000,12715.000000
2023-08,10155.000000,11937.000000,11101.000000
2023-09,10997.000000,11850.000000,8075.000000


In [228]:
df.sample(10)

,order_id,customer_id,order_date,order_amount,platform,category,order_day,first_order_month,day_from_start
2643,2644,466,2023-04-10 11:58:17.328643407,17288.083957,shop,smartphones,2023-04-10,2023-04,6
6752,6753,384,2023-09-12 04:29:08.150964920,5617.025298,desktop,laptops,2023-09-12,2023-03,170
5977,5978,53,2023-08-14 00:16:01.003897708,14371.164610,mobile,smartphones,2023-08-14,2023-01,223
4319,4320,607,2023-06-12 14:15:02.823462306,11980.058857,shop,laptops,2023-06-12,2023-06,0
3352,3353,122,2023-05-07 04:33:30.989637798,2814.337422,mobile,laptops,2023-05-07,2023-01,110
8815,8816,470,2023-11-28 20:25:20.905028996,24542.067240,desktop,laptops,2023-11-28,2023-04,227
7664,7665,491,2023-10-16 12:29:05.413062080,237.106418,mobile,laptops,2023-10-16,2023-04,174
5479,5480,226,2023-07-26 06:19:11.972620972,12320.412871,mobile,smartphones,2023-07-26,2023-02,175
9240,9241,68,2023-12-14 20:24:48.050194888,19509.625037,mobile,watches,2023-12-14,2023-01,342
4819,4820,629,2023-07-01 10:00:17.111892766,502.736722,shop,smartphones,2023-07-01,2023-07,0


In [207]:
df.size

229950